In [190]:
import os
import numpy as np
from astropy import units as u
from astroquery.vizier import Vizier
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.time import Time

In [89]:
Vizier.ROW_LIMIT = -1
dias2021 = Vizier.get_catalogs(catalog="J/MNRAS/504/356/table12")[0]
mask_logage = dias2021['logage'] < 7.7
mask_plx = dias2021['Plx'] > 0.3
dias2021 = dias2021[mask_logage & mask_plx]
dias2021.write('dias2021.tsv', format='ascii.ecsv', overwrite=True)
dias2021[:5]


Cluster,RA_ICRS,DE_ICRS,r50,N,pmRA,e_pmRA,pmDE,e_pmDE,Plx,e_Plx,RV,e_RV,NRV,Dist,e_Dist,logage,e_logage,__Fe_H_,e__Fe_H_,Av,e_Av,FileName,SimbadName,_RA.icrs,_DE.icrs
,deg,deg,deg,,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,km / s,km / s,,pc,pc,log(yr),log(yr),,,mag,mag,,,deg,deg
str17,float64,float64,float32,int16,float32,float32,float32,float32,float32,float32,float64,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,str31,str31,float64,float64
ASCC_107,297.1623,22.0071,0.156,59,-0.144,0.135,-5.158,0.141,1.118,0.055,--,--,--,864,30,7.440,0.121,0.353,0.103,1.372,0.129,clusters1/ASCC_107.dat,[KPR2005] 107,297.1623,22.0071
ASCC_114,324.9790,53.9990,0.180,149,-3.754,0.210,-3.435,0.145,1.063,0.039,--,--,--,911,12,7.632,0.271,0.035,0.078,1.216,0.091,clusters1/ASCC_114.dat,[KPR2005] 114,324.9790,53.9990
ASCC_127,347.1807,64.9151,0.541,113,7.490,0.261,-1.781,0.319,2.618,0.080,-11.267,2.676,16,365,10,7.496,0.131,0.152,0.115,0.668,0.080,clusters1/ASCC_127.dat,[KPR2005] 127,347.1806,64.9151
ASCC_13,78.3057,44.4212,0.564,110,-0.477,0.111,-1.737,0.108,0.899,0.076,--,--,--,1066,26,7.615,0.098,-0.075,0.078,0.915,0.027,clusters1/ASCC_13.dat,[KPR2005] 13,78.3057,44.4212
ASCC_16,81.2025,1.6256,0.367,207,1.363,0.280,0.002,0.274,2.844,0.113,21.308,1.696,12,348,3,7.088,0.061,-0.062,0.069,0.224,0.045,clusters1/ASCC_16.dat,[KPR2005] 16,81.2025,1.6256


In [198]:
class clusterDias:
    def __init__(self, name:str) -> None:
        #download the catalog if not already in the cwds
        if os.path.exists('./dias2021.tsv'):
            dias2021 = Table.read('dias2021.tsv', format='ascii.ecsv')
        else:
            Vizier.ROW_LIMIT = -1
            dias2021 = Vizier.get_catalogs(catalog="J/MNRAS/504/356/table12")[0]
            mask_logage = dias2021['logage'] < 7.7
            mask_plx = dias2021['Plx'] > 0.3
            dias2021 = dias2021[mask_logage & mask_plx]
        #select the row from Dias catalog
        cluster_row = dias2021[dias2021['Cluster'] == name][0]
        #cluster parameters from the row
        self.name = name
        self.all = cluster_row
        self.r50 = cluster_row['r50']*u.deg
        self.N = cluster_row['N']
        self.skycoord = SkyCoord(ra=cluster_row['RA_ICRS']*u.deg,
                                 dec=cluster_row['DE_ICRS']*u.deg,
                                 distance=cluster_row['Dist']*u.pc,
                                 pm_ra_cosdec=cluster_row['pmRA']*u.mas/u.yr,
                                 pm_dec=cluster_row['pmDE']*u.mas/u.yr,
                                 obstime=(Time('J2000')+1*u.Myr))
        self.ra = self.skycoord.ra
        self.dec = self.skycoord.dec
        self.distance,self.e_distance = self.skycoord.distance,cluster_row['e_Dist']*u.pc
        self.pm_ra_cosdec, self.e_pm_ra_cosdec = self.skycoord.pm_ra_cosdec,cluster_row['e_pmRA']*u.mas/u.yr
        self.pm_dec, self.e_pm_dec = self.skycoord.pm_dec, cluster_row['e_pmDE']*u.mas/u.yr
        self.members = (Table.read(f'./Clusters_Dias/{self.name}.dat', format='ascii.tab'))[2:] #first two rows removed
        self.members_list = list(self.members['Source'].astype(np.int64))
        # self.membersdr3 = list(self.members['Source'])
        self.Av, self.e_Av = cluster_row['Av']*u.mag,cluster_row['e_Av']*u.mag
        self.logage, self.e_logage = cluster_row['logage'],cluster_row['e_logage']
        self.FeH, self.e_FeH = cluster_row['__Fe_H_'],cluster_row['e__Fe_H_']
        self.RV, self.e_RV = cluster_row['RV'],cluster_row['e_RV']
        self.NRV = cluster_row['NRV']

class clusterCG:
    def __init__(self, name:str) -> None:
        self.name = name
        #download the catalog if not already in the cwds
        if os.path.exists('./CG2020.tsv'):
            dias2021 = Table.read('CG2020.tsv', format='ascii.ecsv')
        else:
            Vizier.ROW_LIMIT = -1
            CG2020 = Vizier.get_catalogs(catalog="J/A+A/633/A99/table1")[0]
            mask_logage = CG2020['logage'] < 7.7
            mask_plx = CG2020['Plx'] > 0.3
            CG2020 = CG2020[mask_logage & mask_plx]


In [202]:
CG2020 = Vizier.get_catalogs(catalog="J/A+A/633/A99/table1")[0]
# mask_logage = CG2020['logage'] < 7.7
mask_plx = CG2020['Plx'] > 0.3
CG2020 = CG2020[mask_plx]
CG2020

Cluster,RA_ICRS,DE_ICRS,GLON,GLAT,r50,N,pmRA,pmDE,Plx,dmode,dmode_01,dmode-01,X,Y,Z,Rgc,SimbadName,_RA.icrs,_DE.icrs
,deg,deg,deg,deg,deg,,mas / yr,mas / yr,mas,pc,pc,pc,pc,pc,pc,pc,,deg,deg
str17,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,str26,float64,float64
ASCC_10,51.870,34.981,155.723,-17.770,0.558,71,-1.737,-1.368,1.459,672.0,629.6,720.4,-583.3,263.1,-205.1,8927.2,[KPR2005] 10,51.8700,34.9810
ASCC_101,288.399,36.369,68.028,11.608,0.372,75,0.934,1.288,2.488,397.3,382.2,413.8,145.6,360.9,79.9,8202.3,[KPR2005] 101,288.3990,36.3690
ASCC_105,295.548,27.366,62.825,2.063,0.648,127,1.464,-1.635,1.783,551.8,522.9,584.0,251.8,490.5,19.9,8103.0,[KPR2005] 105,295.5480,27.3660
ASCC_107,297.164,21.987,58.904,-1.901,0.174,59,-0.155,-5.156,1.109,878.5,807.6,963.1,453.5,751.9,-29.2,7922.3,[KPR2005] 107,297.1640,21.9870
ASCC_108,298.306,39.349,74.378,6.074,0.537,230,-0.519,-1.690,0.838,1154.0,1034.6,1304.5,309.0,1105.1,122.1,8106.7,[KPR2005] 108,298.3060,39.3490
ASCC_11,53.056,44.856,150.546,-9.224,0.312,276,0.926,-3.030,1.141,854.5,787.2,934.3,-734.4,414.7,-137.0,9083.9,[KPR2005] 11,53.0560,44.8560
ASCC_110,300.742,33.528,70.411,1.378,0.203,70,0.271,-3.132,0.497,1902.2,1598.2,2349.1,637.6,1791.6,45.7,7908.1,[KPR2005] 110,300.7420,33.5280
ASCC_111,302.891,37.515,74.714,2.056,0.537,156,-1.150,-1.524,1.166,836.9,772.3,913.4,220.5,806.8,30.0,8159.5,[KPR2005] 111,302.8910,37.5150
